In [ ]:
import os, sys
import tensorflow as tf
import tensorlayer as tl
import numpy as np
import network, img_io
import matplotlib.pyplot as plt

## 1. Import LDR images 
these HDR images are derived from HDR image dataset from google. 
the code used for this procedure is function from MATLAB


## 2. Preprocess the images 
Convert the images into smaples??? or just image before the filters 

## 3. create input pipeline for generating training/testing batches

In [ ]:
# this can be either X_train/y_train or X_test/y_test, so we make a placeholder that we can feed into:
X_train_or_test = tf.placeholder(tf.complex64, [None, 28, 28], name='input_image')
y_train_or_test = tf.placeholder(tf.int32, [None], name='image_label')
batch_size = 32

# create a tf dataset, from which we can generate batches
dataset = tf.data.Dataset.from_tensor_slices((X_train_or_test, y_train_or_test))
dataset = dataset.batch(batch_size).repeat(None)
batch_generator = dataset.make_initializable_iterator()
X_batch, y_batch = batch_generator.get_next()  # batches symbolically generated

In [ ]:
X_batch

## 4. create three pixel-level filters 

## 5. generate the filtered images


## 6. propagate back to the image plane: filtered LDR 


 ## 7. process the filtered image through a Physical CNN to optimize filters

- ### Pass through Digital layer to get HDR output image.
- ### use the same loss as the Digital layer. 
loss is illumination + reflectance loss
I just need revise the hdrcnn_predict.py into a function, which return the HDR image and the loss?

revise get_final to return not only the HDR imgae, but also the cost. 

or should I define my own loss? 

FLAGS.sep_loss is the flag to separate loss into illumination and reflectance


### Cost function 

In [4]:
# Learning parameters

tf.flags.DEFINE_bool("sep_loss",            "true",    "Use illumination + reflectance loss")


NameError: name 'tf' is not defined

In [ ]:
import scipy.stats as st[[[]]]

In [3]:
# Loss separated into illumination and reflectance terms
if FLAGS.sep_loss:
    y_log_ = tf.log(y_+eps)  
    x_log = tf.log(tf.pow(x, 2.0)+eps)

    # Luminance
    lum_kernel = np.zeros((1, 1, 3, 1))
    lum_kernel[:, :, 0, 0] = 0.213
    lum_kernel[:, :, 1, 0] = 0.715
    lum_kernel[:, :, 2, 0] = 0.072
    y_lum_lin_ = tf.nn.conv2d(y_, lum_kernel, [1, 1, 1, 1], padding='SAME')
    y_lum_lin = tf.nn.conv2d(tf.exp(y)-eps, lum_kernel, [1, 1, 1, 1], padding='SAME')
    x_lum_lin = tf.nn.conv2d(x, lum_kernel, [1, 1, 1, 1], padding='SAME')

    # Log luminance
    y_lum_ = tf.log(y_lum_lin_ + eps)
    y_lum = tf.log(y_lum_lin + eps)
    x_lum = tf.log(x_lum_lin + eps)

    # Gaussian kernel
    nsig = 2
    filter_size = 13
    interval = (2*nsig+1.)/(filter_size)
    ll = np.linspace(-nsig-interval/2., nsig+interval/2., filter_size+1)
    kern1d = np.diff(st.norm.cdf(ll))
    kernel = kernel_raw/kernel_raw.sum()

    # Illumination, approximated by means of Gaussian filtering
    weights_g = np.zeros((filter_size, filter_size, 1, 1))
    weights_g[:, :, 0, 0] = kernel
    y_ill_ = tf.nn.conv2d(y_lum_, weights_g, [1, 1, 1, 1], padding='SAME')
    y_ill = tf.nn.conv2d(y_lum, weights_g, [1, 1, 1, 1], padding='SAME')
    x_ill = tf.nn.conv2d(x_lum, weights_g, [1, 1, 1, 1], padding='SAME')

    # Reflectance
    y_refl_ = y_log_ - tf.tile(y_ill_, [1,1,1,3])
    y_refl = y - tf.tile(y_ill, [1,1,1,3])
    x_refl = x - tf.tile(x_ill, [1,1,1,3])

    cost =              tf.reduce_mean( ( FLAGS.lambda_ir*tf.square( tf.subtract(y_ill, y_ill_) ) + (1.0-FLAGS.lambda_ir)*tf.square( tf.subtract(y_refl, y_refl_) ) )*msk )
    cost_input_output = tf.reduce_mean( ( FLAGS.lambda_ir*tf.square( tf.subtract(x_ill, y_ill_) ) + (1.0-FLAGS.lambda_ir)*tf.square( tf.subtract(x_refl, y_refl_) ) )*msk )
else:
    cost =              tf.reduce_mean( tf.square( tf.subtract(y, tf.log(y_+eps) )*msk ) )
    cost_input_output = tf.reduce_mean( tf.square( tf.subtract(tf.log(y_+eps), tf.log(tf.pow(x, 2.0)+eps) )*msk ) );


NameError: name 'FLAGS' is not defined

## 8. Train

## 9. Results